## Dataset Preprocessing

In [1]:
import glob
import os

import numpy as np
from PIL import Image
from skimage.io import imread
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch

print(torch.__version__)

c:\jason\new magang\DUCK-Net\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.12.1+cu113


In [ ]:
img_height = 352
img_width = 352

image_path = "C:\\jason\\new magang\\DUCK-Net\\kvasir\\images\\cju887ftknop008177nnjt46y.jpg"
mask_path = image_path.replace("images", "masks")

image = imread(image_path)
mask_ = imread(mask_path)

In [ ]:
X_train = np.zeros((1, img_height, img_width, 3), dtype=np.float32)
Y_train = np.zeros((1, img_height, img_width), dtype=np.uint8)

In [ ]:
mask = np.zeros((img_height, img_width), dtype=np.bool_)

pillow_image = Image.fromarray(image)

pillow_image = pillow_image.resize((img_height, img_width))
image = np.array(pillow_image)

In [ ]:
X_train = image / 255

pillow_mask = Image.fromarray(mask_)
pillow_mask = pillow_mask.resize((img_height, img_width), resample=Image.LANCZOS)
mask_ = np.array(pillow_mask)
mask_ = np.mean(mask_, axis=2)

In [ ]:
for i in range(img_height):
    for j in range(img_width):
        if mask_[i, j] >= 127:
                    mask[i, j] = 1

Y_train = mask
Y_train = np.expand_dims(Y_train, axis=-1)

In [ ]:
# X_train = X_train.astype("uint8")
plt.imshow(X_train)

## Testing the Custom Convolutional Layer of DUCK-Net

In [2]:
import os
import sys
import torch

pathNow = os.path.abspath(os.path.join(os.path.dirname('__file__'), '../CustomLayers'))
print(pathNow)
if pathNow not in sys.path:
    sys.path.append(pathNow)
print('_______________________________')

from PyTorchConvBlock2D import ConvBlock2D

input_channels = 3
starting_filters = 64

# Instantiate the ConvBlock2D with filters 64 and block type 'duckv2'
conv_block = ConvBlock2D(input_channels, starting_filters, 'duckv2', repeat=2)
conv_block2 = ConvBlock2D(starting_filters, starting_filters*2, 'duckv2', repeat=2)

# Create a random input tensor with shape (batch_size, channels, height, width)
# Assuming channels are 64 to match the filters in the ConvBlock2D
input_tensor = torch.randn(1, 3, 256, 256)  # Example with batch_size=1, channels=64, height=32, width=32

# Pass the input tensor through the module
output_tensor = conv_block(input_tensor)
output_tensor = conv_block2(output_tensor)
# Print the shape of the output tensor
print("Output shape:", output_tensor.shape)

c:\jason\new magang\DUCK-Net\CustomLayers
_______________________________
Output shape: torch.Size([1, 128, 256, 256])


## Testing DUCK-Net

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from PyTorchConvBlock2D import ConvBlock2D

class DuckNet(nn.Module):
    def __init__(self, img_height, img_width, input_channels, out_classes, starting_filters):
        super(DuckNet, self).__init__()

        self.starting_filters = starting_filters

        self.conv1 = nn.Conv2d(input_channels, starting_filters * 2, kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(starting_filters * 2, starting_filters * 4, kernel_size=2, stride=2, padding=0)
        self.conv3 = nn.Conv2d(starting_filters * 4, starting_filters * 8, kernel_size=2, stride=2, padding=0)
        self.conv4 = nn.Conv2d(starting_filters * 8, starting_filters * 16, kernel_size=2, stride=2, padding=0)
        self.conv5 = nn.Conv2d(starting_filters * 16, starting_filters * 32, kernel_size=2, stride=2, padding=0)

        self.t0 = ConvBlock2D(input_channels, starting_filters, 'duckv2', repeat=1)

        self.l1i = nn.Conv2d(starting_filters, starting_filters * 2, kernel_size=2, stride=2, padding=0)
        self.t1 = ConvBlock2D(starting_filters * 2, starting_filters * 2, 'duckv2', repeat=1)

        self.l2i = nn.Conv2d(starting_filters * 2, starting_filters * 4, kernel_size=2, stride=2, padding=0)
        self.t2 = ConvBlock2D(starting_filters * 4, starting_filters * 4, 'duckv2', repeat=1)

        self.l3i = nn.Conv2d(starting_filters * 4, starting_filters * 8, kernel_size=2, stride=2, padding=0)
        self.t3 = ConvBlock2D(starting_filters * 8, starting_filters * 8, 'duckv2', repeat=1)

        self.l4i = nn.Conv2d(starting_filters * 8, starting_filters * 16, kernel_size=2, stride=2, padding=0)
        self.t4 = ConvBlock2D(starting_filters * 16, starting_filters * 16, 'duckv2', repeat=1)

        self.l5i = nn.Conv2d(starting_filters * 16, starting_filters * 32, kernel_size=2, stride=2, padding=0)
        self.t51 = ConvBlock2D(starting_filters * 32, starting_filters * 32, 'resnet', repeat=2)
        self.t53 = ConvBlock2D(starting_filters * 32, starting_filters * 16, 'resnet', repeat=2)

        self.l5o = nn.Upsample(scale_factor=2, mode='nearest')
        self.c4 = ConvBlock2D(starting_filters * 16, starting_filters * 8, 'duckv2', repeat=1)

        self.l4o = nn.Upsample(scale_factor=2, mode='nearest')
        self.c3 = ConvBlock2D(starting_filters * 8, starting_filters * 4, 'duckv2', repeat=1)

        self.l3o = nn.Upsample(scale_factor=2, mode='nearest')
        self.c2 = ConvBlock2D(starting_filters * 4, starting_filters * 2, 'duckv2', repeat=1)

        self.l2o = nn.Upsample(scale_factor=2, mode='nearest')
        self.c1 = ConvBlock2D(starting_filters * 2, starting_filters, 'duckv2', repeat=1)

        self.l1o = nn.Upsample(scale_factor=2, mode='nearest')
        self.c0 = ConvBlock2D(starting_filters, starting_filters, 'duckv2', repeat=1)

        self.output = nn.Conv2d(starting_filters, out_classes, kernel_size=1)

    def forward(self, x):
        p1 = self.conv1(x)
        p2 = self.conv2(p1)
        p3 = self.conv3(p2)
        p4 = self.conv4(p3)
        p5 = self.conv5(p4)

        t0 = self.t0(x)

        l1i = self.l1i(t0)
        s1 = l1i + p1
        t1 = self.t1(s1)

        l2i = self.l2i(t1)
        s2 = l2i + p2
        t2 = self.t2(s2)

        l3i = self.l3i(t2)
        s3 = l3i + p3
        t3 = self.t3(s3)

        l4i = self.l4i(t3)
        s4 = l4i + p4
        t4 = self.t4(s4)

        l5i = self.l5i(t4)
        s5 = l5i + p5
        t51 = self.t51(s5)
        t53 = self.t53(t51)

        l5o = self.l5o(t53)
        c4 = l5o + t4
        q4 = self.c4(c4)

        l4o = self.l4o(q4)
        c3 = l4o + t3
        q3 = self.c3(c3)

        l3o = self.l3o(q3)
        c2 = l3o + t2
        q6 = self.c2(c2)

        l2o = self.l2o(q6)
        c1 = l2o + t1
        q1 = self.c1(c1)

        l1o = self.l1o(q1)
        c0 = l1o + t0
        z1 = self.c0(c0)

        output = self.output(z1)

        return output

# Instantiate the model
img_height=256
img_width = 256 
input_channels = 3
out_classes=1
starting_filters = 64

model = DuckNet(img_height, img_width, input_channels, out_classes, starting_filters)

# Generate a random input tensor
input_tensor = torch.randn(1, input_channels, img_height, img_width)

# Forward pass
output_tensor = model(input_tensor)

print("Output tensor shape:", output_tensor.shape)

Output tensor shape: torch.Size([1, 1, 256, 256])
